In [1]:
from corus import load_wiki
from bs4 import BeautifulSoup
import re
from gensim.utils import tokenize
from razdel import tokenize
import string
import nltk
from nltk.corpus import stopwords
import pymorphy2
from spellchecker import SpellChecker

# Получение данных

In [2]:
path = 'ruwiki-latest-pages-articles.xml.bz2'
records = load_wiki(path)
dataset = list(records)[0].text
dataset[:1000]

'Литва\n\nЛитва́ (), официальное название\xa0— Лито́вская Респу́блика ()\xa0— государство, расположенное в северо-восточной части Европы. Столица страны\xa0— Вильнюс.Площадь\xa0— км². Протяжённость с севера на юг\xa0— 280\xa0км, а с запада на восток\xa0— 370 км. Население составляет человек\xa0— по этим показателям является крупнейшим прибалтийским государством. Имеет выход к Балтийскому морю, расположена на его восточном побережье. Береговая линия составляет всего 99 км (наименьший показатель среди прибалтийских государств). На севере граничит с Латвией, на юго-востоке\xa0— с Белоруссией, на юго-западе\xa0— с Польшей и Калининградской областью России.Член ООН с 1991 года, ЕС и НАТО\xa0— с 2004 года, ОЭСР\xa0— с мая 2018 года. Входит в Шенгенскую зону и Еврозону.== Географические данные ==Поверхность\xa0— равнинная со следами древнего оледенения. Поля и луга занимают 57\xa0% территории, леса и кустарники\xa0— 30\xa0%, болота\xa0— 6\xa0%, внутренние воды\xa0— 1\xa0%.Высшая точка\xa0— 29

# Очистка

In [3]:
soup = BeautifulSoup(dataset, 'html.parser')
for data in soup(['style', 'script']):
  data.decompose()
dataset = ' '.join(soup.stripped_strings)
dataset[:1000]

'Литва\n\nЛитва́ (), официальное название\xa0— Лито́вская Респу́блика ()\xa0— государство, расположенное в северо-восточной части Европы. Столица страны\xa0— Вильнюс.Площадь\xa0— км². Протяжённость с севера на юг\xa0— 280\xa0км, а с запада на восток\xa0— 370 км. Население составляет человек\xa0— по этим показателям является крупнейшим прибалтийским государством. Имеет выход к Балтийскому морю, расположена на его восточном побережье. Береговая линия составляет всего 99 км (наименьший показатель среди прибалтийских государств). На севере граничит с Латвией, на юго-востоке\xa0— с Белоруссией, на юго-западе\xa0— с Польшей и Калининградской областью России.Член ООН с 1991 года, ЕС и НАТО\xa0— с 2004 года, ОЭСР\xa0— с мая 2018 года. Входит в Шенгенскую зону и Еврозону.== Географические данные ==Поверхность\xa0— равнинная со следами древнего оледенения. Поля и луга занимают 57\xa0% территории, леса и кустарники\xa0— 30\xa0%, болота\xa0— 6\xa0%, внутренние воды\xa0— 1\xa0%.Высшая точка\xa0— 29

In [4]:
pattern = r'\xa0'
dataset = re.sub(pattern, '', dataset)
pattern = r'\n'
dataset = re.sub(pattern, ' ', dataset)
dataset[:1000]

'Литва  Литва́ (), официальное название— Лито́вская Респу́блика ()— государство, расположенное в северо-восточной части Европы. Столица страны— Вильнюс.Площадь— км². Протяжённость с севера на юг— 280км, а с запада на восток— 370 км. Население составляет человек— по этим показателям является крупнейшим прибалтийским государством. Имеет выход к Балтийскому морю, расположена на его восточном побережье. Береговая линия составляет всего 99 км (наименьший показатель среди прибалтийских государств). На севере граничит с Латвией, на юго-востоке— с Белоруссией, на юго-западе— с Польшей и Калининградской областью России.Член ООН с 1991 года, ЕС и НАТО— с 2004 года, ОЭСР— с мая 2018 года. Входит в Шенгенскую зону и Еврозону.== Географические данные ==Поверхность— равнинная со следами древнего оледенения. Поля и луга занимают 57% территории, леса и кустарники— 30%, болота— 6%, внутренние воды— 1%.Высшая точка— 293,84 м над уровнем моря— холм Аукштояс () (или Аукштасис калнас ()) в юго-восточной ча

В датасете много имен собственных, так что для каких-то задач приведение к нижнему регистру может помешать, но все равно сделаем.

In [5]:
dataset = dataset.lower()
dataset[:1000]

'литва  литва́ (), официальное название— лито́вская респу́блика ()— государство, расположенное в северо-восточной части европы. столица страны— вильнюс.площадь— км². протяжённость с севера на юг— 280км, а с запада на восток— 370 км. население составляет человек— по этим показателям является крупнейшим прибалтийским государством. имеет выход к балтийскому морю, расположена на его восточном побережье. береговая линия составляет всего 99 км (наименьший показатель среди прибалтийских государств). на севере граничит с латвией, на юго-востоке— с белоруссией, на юго-западе— с польшей и калининградской областью россии.член оон с 1991 года, ес и нато— с 2004 года, оэср— с мая 2018 года. входит в шенгенскую зону и еврозону.== географические данные ==поверхность— равнинная со следами древнего оледенения. поля и луга занимают 57% территории, леса и кустарники— 30%, болота— 6%, внутренние воды— 1%.высшая точка— 293,84 м над уровнем моря— холм аукштояс () (или аукштасис калнас ()) в юго-восточной ча

# Токенизация

In [6]:
data = [word for word in tokenize(dataset)]
print(data[:100])

[Substring(0, 5, 'литва'), Substring(7, 13, 'литва́'), Substring(14, 15, '('), Substring(15, 16, ')'), Substring(16, 17, ','), Substring(18, 29, 'официальное'), Substring(30, 39, 'название—'), Substring(40, 50, 'лито́вская'), Substring(51, 62, 'респу́блика'), Substring(63, 64, '('), Substring(64, 65, ')'), Substring(65, 66, '—'), Substring(67, 78, 'государство'), Substring(78, 79, ','), Substring(80, 93, 'расположенное'), Substring(94, 95, 'в'), Substring(96, 112, 'северо-восточной'), Substring(113, 118, 'части'), Substring(119, 125, 'европы'), Substring(125, 126, '.'), Substring(127, 134, 'столица'), Substring(135, 142, 'страны—'), Substring(143, 150, 'вильнюс'), Substring(150, 151, '.'), Substring(151, 159, 'площадь—'), Substring(160, 163, 'км²'), Substring(163, 164, '.'), Substring(165, 178, 'протяжённость'), Substring(179, 180, 'с'), Substring(181, 187, 'севера'), Substring(188, 190, 'на'), Substring(191, 194, 'юг—'), Substring(195, 198, '280'), Substring(198, 200, 'км'), Substring

In [7]:
data = [w.text for w in list(tokenize(dataset))]
print(data[:100])

['литва', 'литва́', '(', ')', ',', 'официальное', 'название—', 'лито́вская', 'респу́блика', '(', ')', '—', 'государство', ',', 'расположенное', 'в', 'северо-восточной', 'части', 'европы', '.', 'столица', 'страны—', 'вильнюс', '.', 'площадь—', 'км²', '.', 'протяжённость', 'с', 'севера', 'на', 'юг—', '280', 'км', ',', 'а', 'с', 'запада', 'на', 'восток—', '370', 'км', '.', 'население', 'составляет', 'человек—', 'по', 'этим', 'показателям', 'является', 'крупнейшим', 'прибалтийским', 'государством', '.', 'имеет', 'выход', 'к', 'балтийскому', 'морю', ',', 'расположена', 'на', 'его', 'восточном', 'побережье', '.', 'береговая', 'линия', 'составляет', 'всего', '99', 'км', '(', 'наименьший', 'показатель', 'среди', 'прибалтийских', 'государств', ')', '.', 'на', 'севере', 'граничит', 'с', 'латвией', ',', 'на', 'юго-востоке—', 'с', 'белоруссией', ',', 'на', 'юго-западе—', 'с', 'польшей', 'и', 'калининградской', 'областью', 'россии', '.']


Razdel справился лучше, чем gensim, оставил даты, которые тут важны. Отдельные токены с пунктуацией уберем дальше.

In [8]:
punct = string.punctuation + '—'
data = [w.strip(punct) for w in data if w.strip(punct) != '']
print(data[:100])

['литва', 'литва́', 'официальное', 'название', 'лито́вская', 'респу́блика', 'государство', 'расположенное', 'в', 'северо-восточной', 'части', 'европы', 'столица', 'страны', 'вильнюс', 'площадь', 'км²', 'протяжённость', 'с', 'севера', 'на', 'юг', '280', 'км', 'а', 'с', 'запада', 'на', 'восток', '370', 'км', 'население', 'составляет', 'человек', 'по', 'этим', 'показателям', 'является', 'крупнейшим', 'прибалтийским', 'государством', 'имеет', 'выход', 'к', 'балтийскому', 'морю', 'расположена', 'на', 'его', 'восточном', 'побережье', 'береговая', 'линия', 'составляет', 'всего', '99', 'км', 'наименьший', 'показатель', 'среди', 'прибалтийских', 'государств', 'на', 'севере', 'граничит', 'с', 'латвией', 'на', 'юго-востоке', 'с', 'белоруссией', 'на', 'юго-западе', 'с', 'польшей', 'и', 'калининградской', 'областью', 'россии', 'член', 'оон', 'с', '1991', 'года', 'ес', 'и', 'нато', 'с', '2004', 'года', 'оэср', 'с', 'мая', '2018', 'года', 'входит', 'в', 'шенгенскую', 'зону', 'и']


# Удаление стоп-слов

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
sw = stopwords.words('russian')
print(sw)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [11]:
data = [w for w in data if w not in sw]

# Лемматизация

In [12]:
morph = pymorphy2.MorphAnalyzer()
data = [morph.parse(i)[0].normal_form for i in data]
print(data[:100])

['литва', 'литва́', 'официальный', 'название', 'лито́вский', 'респу́блик', 'государство', 'расположить', 'северо-восточный', 'часть', 'европа', 'столица', 'страна', 'вильнюс', 'площадь', 'км²', 'протяжённость', 'север', 'юг', '280', 'км', 'запад', 'восток', '370', 'км', 'население', 'составлять', 'человек', 'это', 'показатель', 'являться', 'крупный', 'прибалтийский', 'государство', 'иметь', 'выход', 'балтийский', 'море', 'расположить', 'восточный', 'побережье', 'береговой', 'линия', 'составлять', '99', 'км', 'малый', 'показатель', 'среди', 'прибалтийский', 'государство', 'север', 'граничить', 'латвия', 'юго-восток', 'белоруссия', 'юго-запад', 'польша', 'калининградский', 'область', 'россия', 'член', 'оон', '1991', 'год', 'ес', 'нато', '2004', 'год', 'оэср', 'май', '2018', 'год', 'входить', 'шенгенский', 'зона', 'еврозона', 'географический', 'дать', 'поверхность', 'равнинный', 'след', 'древний', 'оледенение', 'поле', 'луг', 'занимать', '57', 'территория', 'лес', 'кустарник', '30', 'боло

# Проверка корректности

In [13]:
spell = SpellChecker(language='ru')
print(spell.unknown(data))

{'еврозона', 'кремнёвый', 'разный', 'упомянуть', 'the', 'упадок', 'охватывать', 'серп', 'подчинить', 'v', 'обязаться', 'примечание', 'оэср', 'всеобщий', 'нато', 'последовать', 'вассал', 'балтийский', 'оппозиционный', 'славянский', '»', 'панъевропейский', '1392—1430', 'антикризисный', 'белоруссия', 'луг', 'a', 'взаимопомощь', 'раздельный', 'уравнивать', 'миндовг', 'духовенство', 'ягеллон', 'всенародно', 'посполить', 'неотъемлемый', 'н', 'статут', 'двина', 'тыс', 'зарождение', 'деполонизация', 'земельный', 'тариб', 'скотоводство', 'аукштасис', 'аугустинас', 'всеобъемлющий', 'венгерский', '0,10', '978-609-437-163-9', 'гербовый', 'средиземный', 'пятисотлетний', 'калининградский', 'эмиграция', 'проголосовать', '4,1', 'воз', 'равнинный', 'млн', 'самостоятельный', 'становление', 'сложиться', 'грибаускать', 'вооружённый', '2,45', 'xiv', 'галицко-волынский', 'таурагнас', 'цензура', 'остланда', 'окончиться', 'балта', 'географический', 'выстроить', 'вещать', 'конституционный', 'казимир', 'национа

SpellChecker нашел некоторые ошибки, но в основном выдал правильно написанные слова, которые он не знает.